In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date,datetime,timedelta
import gc

In [2]:
pd.options.display.max_columns=None
pd.options.display.max_info_columns=1000

## Read Prospect

In [3]:
df_prospects=pd.read_csv('/home/darshan/dev/AI_ML_Projects/schneider_electric/data/Schneider Electric Data/Email_Enagement_Data/SE Projects Consolidated File(Clean).csv')

/tmp/ipykernel_1290288/591620312.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_prospects=pd.read_csv('/home/darshan/dev/AI_ML_Projects/schneider_electric/data/Schneider Electric Data/Email_Enagement_Data/SE Projects Consolidated File(Clean).csv')


In [4]:
df_prospects.shape

(156043, 29)

In [5]:
df_prospects.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 156043 entries, 0 to 156042
Data columns (total 29 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             156043 non-null  int64  
 1   Email                  156040 non-null  object 
 2   Project Name           156043 non-null  object 
 3   Delivery Status        156043 non-null  object 
 4   Title                  156043 non-null  object 
 5   Level                  142987 non-null  object 
 6   Department             142987 non-null  object 
 7   Contact_City           133182 non-null  object 
 8   Contact_State          116263 non-null  object 
 9   Contact_Zipcode        5183 non-null    object 
 10  Country                155851 non-null  object 
 11  Company City           149115 non-null  object 
 12  Company State          138500 non-null  object 
 13  Company Post Code      83733 non-null   object 
 14  Company Country        154396 non-nu

## Read Email Engagement

In [6]:
df_prospect_engagements=pd.read_csv('/home/darshan/dev/AI_ML_Projects/schneider_electric/data/Schneider Electric Data/Email_Enagement_Data/new_unique_prospect_engagements.csv')

In [7]:
df_prospect_engagements.shape

(4036649, 25)

In [8]:
df_prospect_engagements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4036649 entries, 0 to 4036648
Data columns (total 25 columns):
 #   Column            Dtype  
---  ------            -----  
 0   FirstName         object 
 1   CompanyName       object 
 2   source            object 
 3   header_date       object 
 4   raw_status        object 
 5   status            object 
 6   status_time       object 
 7   reason            object 
 8   status_time_utc   object 
 9   Email             object 
 10  day_of_week       object 
 11  is_clicked_mail   int64  
 12  time_category     object 
 13  hour              float64
 14  segment           object 
 15  sender_email      object 
 16  header_date_utc   object 
 17  is_email_sent     int64  
 18  is_email_clicked  int64  
 19  is_email_opened   int64  
 20  is_responded      int64  
 21  is_bounced        int64  
 22  is_unsubscribed   int64  
 23  is_mail_not_sent  int64  
 24  is_scheduled      int64  
dtypes: float64(1), int64(9), object(15)
memory us

In [9]:
df_prospect_engagements['segment'].isnull().sum()

np.int64(0)

In [10]:
df_prospect_engagements['status'].unique()

array(['EMAIL_SENT', 'BOUNCED', 'EMAIL_NOT_SENT', nan, 'ERROR',
       'SCHEDULED', 'EMAIL_CLICKED', 'EMAIL_OPENED', 'RESPONDED',
       'UNSUBSCRIBED', '-'], dtype=object)

## Extract Engagement data. if any prospect CLICK EMAIL in SEGMENT

In [11]:
df_total_email_clicked_count = df_prospect_engagements.groupby(["Email","segment"])["is_clicked_mail"].sum().reset_index(name='total_clicked_mail')

In [12]:
df_total_email_clicked_count.head(5)

,Email,segment,total_clicked_mail
0,9fake@thomasfellows.com,semiconductor,0
1,Adaku.Ude@sfdph.org,airport,0
2,Ahess@amgen.com,life sciences,0
3,Carlos.Perez@thermofisher.com,life sciences,0
4,CarlosOliverMontejano.Ramirez@magna.com,mexico,0


In [13]:
df_total_prospects_with_clicked = df_total_email_clicked_count[df_total_email_clicked_count["total_clicked_mail"] > 0]

In [14]:
df_total_prospects_with_clicked.shape

(9769, 3)

In [15]:
df_clicked_prospect_engagements = df_prospect_engagements.merge(df_total_prospects_with_clicked,on=["Email","segment"],how="inner")

In [16]:
df_clicked_prospect_engagements.shape

(428146, 26)

In [17]:
df_clicked_prospect_engagements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428146 entries, 0 to 428145
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   FirstName           428146 non-null  object 
 1   CompanyName         428146 non-null  object 
 2   source              428146 non-null  object 
 3   header_date         424301 non-null  object 
 4   raw_status          424785 non-null  object 
 5   status              424784 non-null  object 
 6   status_time         418459 non-null  object 
 7   reason              1241 non-null    object 
 8   status_time_utc     418459 non-null  object 
 9   Email               428146 non-null  object 
 10  day_of_week         418459 non-null  object 
 11  is_clicked_mail     428146 non-null  int64  
 12  time_category       428146 non-null  object 
 13  hour                418459 non-null  float64
 14  segment             428146 non-null  object 
 15  sender_email        428146 non-nul

In [18]:
del df_total_email_clicked_count

In [19]:
del df_total_prospects_with_clicked

In [20]:
gc.collect()

0

## Repair & Remove invalid Email Engagement

In [21]:
df_clicked_prospect_engagements['status'].value_counts()

status
EMAIL_SENT        385957
EMAIL_CLICKED      24728
EMAIL_NOT_SENT      7263
EMAIL_OPENED        3595
SCHEDULED           1856
BOUNCED             1095
UNSUBSCRIBED         155
ERROR                130
RESPONDED              4
-                      1
Name: count, dtype: int64

In [22]:
df_clicked_prospect_engagements = df_clicked_prospect_engagements[df_clicked_prospect_engagements['status'] != '-']

In [23]:
df_clicked_prospect_engagements['status'] = df_clicked_prospect_engagements['status'].replace('RESPONDED', 'EMAIL_OPENED')

In [24]:
df_clicked_prospect_engagements['status'].value_counts()

status
EMAIL_SENT        385957
EMAIL_CLICKED      24728
EMAIL_NOT_SENT      7263
EMAIL_OPENED        3599
SCHEDULED           1856
BOUNCED             1095
UNSUBSCRIBED         155
ERROR                130
Name: count, dtype: int64

## Add some column use for fetaure creation

In [25]:
df_clicked_prospect_engagements['status_time_utc']=pd.to_datetime(df_clicked_prospect_engagements['status_time_utc'],utc=True)

In [26]:
df_clicked_prospect_engagements.sort_values(by=['Email', 'segment', 'status_time_utc'], ascending=True, inplace=True)

In [27]:
df_prospect_first_clicked_mail_data= df_clicked_prospect_engagements[df_clicked_prospect_engagements['status']=='EMAIL_CLICKED'].groupby(['Email', 'segment']).first().reset_index()
df_prospect_first_clicked_mail_data= df_prospect_first_clicked_mail_data[['Email', 'segment', 'status_time_utc']]
df_prospect_first_clicked_mail_data.columns=['Email', 'segment', 'first_email_clicked_date']

In [28]:
df_prospect_first_opened_mail_data= df_clicked_prospect_engagements[df_clicked_prospect_engagements['status']=='EMAIL_OPENED'].groupby(['Email', 'segment']).first().reset_index()
df_prospect_first_opened_mail_data= df_prospect_first_opened_mail_data[['Email', 'segment', 'status_time_utc']]
df_prospect_first_opened_mail_data.columns=['Email', 'segment', 'first_email_opened_date']

In [29]:
df_clicked_prospect_engagements= df_clicked_prospect_engagements.merge(df_prospect_first_clicked_mail_data,on=['Email', 'segment'],how='left')
df_clicked_prospect_engagements= df_clicked_prospect_engagements.merge(df_prospect_first_opened_mail_data,on=['Email', 'segment'],how='left')

In [30]:

df_clicked_prospect_engagements['first_email_opened_date']= df_clicked_prospect_engagements['first_email_opened_date'].fillna(df_clicked_prospect_engagements['first_email_clicked_date'])

In [31]:
del df_prospect_first_clicked_mail_data

In [32]:
del df_prospect_first_opened_mail_data

In [33]:
gc.collect()

0

In [50]:
df_clicked_prospect_engagements.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428145 entries, 0 to 428144
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   FirstName                 428145 non-null  object             
 1   CompanyName               428145 non-null  object             
 2   source                    428145 non-null  object             
 3   header_date               424300 non-null  object             
 4   raw_status                424784 non-null  object             
 5   status                    424783 non-null  object             
 6   status_time               418459 non-null  object             
 7   reason                    1241 non-null    object             
 8   status_time_utc           418459 non-null  datetime64[ns, UTC]
 9   Email                     428145 non-null  object             
 10  day_of_week               418459 non-null  object             
 11  

In [51]:
df_clicked_prospect_engagements.to_csv('/home/darshan/dev/AI_ML_Projects/schneider_electric/data/Schneider Electric Data/Email_Enagement_Data/click_email_prospect.csv')

## FEATURE ENGINEERING